<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D9%85%D8%A7%D9%83%D8%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """حساب مؤشر MACD"""
    data['EMA_short'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

# استخدام الدالة
data = calculate_macd(data)
ذي

In [ ]:
def calculate_bollinger_bands(data, window=20, num_std_dev=2):
    """حساب بولينجر باندز."""
    try:
        # حساب المتوسط المتحرك
        data['Middle_Band'] = data['close'].rolling(window=window).mean()

        # حساب الانحراف المعياري
        data['Std_Dev'] = data['close'].rolling(window=window).std()

        # حساب الحد العلوي والحد السفلي
        data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
        data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)

        return data
    except Exception as e:
        print(f"An error occurred while calculating Bollinger Bands: {e}")
        return data


In [ ]:
data = get_and_clean_gold_prices()
if data is not None:
    data = calculate_bollinger_bands(data)  # إضافة هذه السطر لحساب بولينجر باندز
    build_strategy(data)


In [ ]:
def calculate_stochastic(data, k_window=14, d_window=3):
    """حساب Stochastic Oscillator"""
    low_min = data['low'].rolling(window=k_window).min()
    high_max = data['high'].rolling(window=k_window).max()
    data['%K'] = 100 * (data['close'] - low_min) / (high_max - low_min)
    data['%D'] = data['%K'].rolling(window=d_window).mean()
    return data


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.72 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.72 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)

In [ ]:
import pandas as pd

# تعريف دالة حساب ADX
def calculate_adx(data, period=14):
    """حساب مؤشر ADX."""
    data['High_Change'] = data['high'].diff()
    data['Low_Change'] = data['low'].diff()

    data['+DM'] = (data['High_Change'] > data['Low_Change']) & (data['High_Change'] > 0) * data['High_Change']
    data['-DM'] = (data['Low_Change'] > data['High_Change']) & (data['Low_Change'] > 0) * data['Low_Change']

    data['TR'] = data[['high', 'low', 'close']].max(axis=1) - data[['high', 'low', 'close']].min(axis=1)
    data['ATR'] = data['TR'].rolling(window=period).mean()

    data['+DI'] = 100 * (data['+DM'].rolling(window=period).sum() / data['ATR'])
    data['-DI'] = 100 * (data['-DM'].rolling(window=period).sum() / data['ATR'])

    data['DX'] = 100 * (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI']))
    data['ADX'] = data['DX'].rolling(window=period).mean()

    return data[['ADX']]

# استخدم الدالة مع بياناتك الخاصة
# تأكد من أن لديك DataFrame بالبيانات المطلوبة (high, low, close)
# adx_data = calculate_adx(your_dataframe)

# طباعة النتائج
# print(adx_data)


In [ ]:
import time

def check_for_high_impact_news():
    """تحقق من وجود أخبار قوية قادمة."""
    # يمكنك استخدام مكتبة مثل requests لجلب الأخبار من موقع مثل Investing.com
    # أو يمكنك استخدام API إذا كان متاحًا.
    # هنا مثال بسيط:
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # ابحث عن الأخبار ذات التأثير العالي
    high_impact_news = soup.select('div.high-impact-news')  # هذا افتراض
    return len(high_impact_news) > 0

# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    while True:
        data = get_and_clean_gold_prices()
        if data is not None:
            build_strategy(data)

        # تحقق من وجود أخبار قوية
        if check_for_high_impact_news():
            time.sleep(30)  # الانتظار لمدة 30 ثانية قبل التكرار
        else:
            time.sleep(60)  # الانتظار لمدة دقيقة قبل التكرار


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على أقرب سعر وصل إليه السعر نحو الهدف بعد سعر افتتاح الصفقة."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # الحصول على أسعار التاريخية بعد فتح الصفقة
            historical_prices = get_historical_prices(symbol, entry_price)

            # تحديد أقرب سعر وصل إليه السعر نحو تيك بروفت
            closest_price = max([price for price in historical_prices if price >= entry_price], default=entry_price)

            distance_to_tp = take_profit - entry_price

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if closest_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if closest_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if closest_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)

def get_historical_prices(symbol, entry_price):
    """استرجاع الأسعار التاريخية بعد سعر افتتاح الصفقة."""
    # يمكنك تعديل هذه الدالة لتناسب احتياجاتك، مثل استخدام `copy_rates_from_pos`
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M15, 0, 1000)
    prices = [rate['close'] for rate in rates if rate['time'] >= entry_price]
    return prices


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # تحقق من عدد الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol="XAUUSD")
        open_positions_count = len(open_positions)

        # إذا كان عدد الصفقات المفتوحة 7، لا تفتح أي صفقات جديدة
        if open_positions_count >= 7:
            print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7). لا يمكن فتح صفقات جديدة.")
            return

        # تحديد الاتجاه العام باستخدام المتوسط المتحرك
        current_ma = data['MA'].iloc[-1]
        previous_ma = data['MA'].iloc[-2]

        # تحقق مما إذا كان الاتجاه قد تغير
        new_trend = "bullish" if current_ma > previous_ma else "bearish"

        current_positions = "buy" if open_positions and open_positions[0].type == 0 else "sell"

        # إذا كان الاتجاه قد تغير، أغلق الصفقات الحالية وافتح صفقات جديدة
        if (new_trend == "bullish" and current_positions == "sell") or (
                new_trend == "bearish" and current_positions == "buy"):
            # إغلاق الصفقات المفتوحة
            for position in open_positions:
                request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "position": position.ticket,
                    "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                    "volume": position.volume,
                    "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick(
                        "XAUUSD").ask,
                    "deviation": 10,
                    "magic": 234000,
                    "comment": "Closing position..."
                }
                result = mt5.order_send(request)

                if result is None or (hasattr(result, 'retcode') and result.retcode != mt5.TRADE_RETCODE_DONE):
                    print(f"فشل في إغلاق الصفقة: {result.retcode if result else 'Unknown error'}")
                else:
                    print(f"تم إغلاق الصفقة بنجاح: {position.ticket}")

        # فتح صفقة جديدة بناءً على الاتجاه الجديد
        if open_positions_count < 7:  # تحقق من العدد قبل فتح صفقة جديدة
            order_type = "buy" if new_trend == "bullish" else "sell"
            volume = calculate_lot_size(account_balance=mt5.account_info().balance)
            print(f"Calculated volume: {volume}")

            price = mt5.symbol_info_tick("XAUUSD").ask if order_type == "buy" else mt5.symbol_info_tick("XAUUSD").bid

            # تعيين SL و TP
            sl = price - 10 if order_type == "buy" else price + 10
            tp = price + 20 if order_type == "buy" else price - 20

            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": "XAUUSD",
                "volume": volume,
                "type": 0 if order_type == "buy" else 1,  # 0 للشراء و 1 للبيع
                "price": price,
                "sl": sl,
                "tp": tp,
                "deviation": 10,
                "magic": 234000,
                "comment": "Opening new position..."
            }

            result = mt5.order_send(request)

            if result is None or (hasattr(result, 'retcode') and result.retcode != mt5.TRADE_RETCODE_DONE):
                print(f"فشل في فتح الصفقة: {result.retcode if result else 'Unknown error'}")
            else:
                print(f"تم فتح الصفقة بنجاح: {order_type} {volume} lots من XAUUSD عند {price}")

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)
        calculate_stochastic(data)
        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # استخدام تحليل المشاعر لتعديل استراتيجيتك
        if average_sentiment > 0.1 and open_positions_count < 7:
            place_order("XAUUSD", "buy", volume=0.01)
        elif average_sentiment < -0.1 and open_positions_count < 7:
            place_order("XAUUSD", "sell", volume=0.01)

        # تأمين الصفقة إذا كانت هناك خسارة كبيرة
        if check_open_positions("XAUUSD"):
            hedge_position_with_retrace_limit("XAUUSD")  # تأمين الصفقة
            set_break_even("XAUUSD")  # تعيين بريك إيفن

            # إغلاق الصفقات الرابحة إذا كانت مجموع الأرباح تعادل ثلاث أرباع مجموع الخسائر
            positions = mt5.positions_get(symbol="XAUUSD")
            total_profit = sum(position.profit for position in positions if position.profit > 0)
            total_loss = sum(-position.profit for position in positions if position.profit < 0)

            # تحقق من الشرط
            if total_profit >= (3 / 4) * total_loss and total_loss > 0:  # تأكد من وجود خسائر
                for position in positions:
                    if position.profit > 0:  # إغلاق الصفقة الرابحة
                        request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": position.ticket,
                            "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                            "volume": position.volume,
                            "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick(
                                "XAUUSD").ask,
                            "deviation": 10,
                            "magic": 234000,
                            "comment": "Closing profitable position",
                            "type_time": mt5.ORDER_TIME_GTC,
                            "type_filling": mt5.ORDER_FILLING_IOC,
                        }
                        result = mt5.order_send(request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"تم إغلاق الصفقة الرابحة لـ {position.ticket}.")

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        last_row = data.iloc[-1]
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        if open_positions_count < 7:  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""

    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # تحديد السعر الحالي بناءً على نوع الصفقة
            if position.type == mt5.ORDER_BUY:
                current_price = mt5.symbol_info_tick(symbol).bid
                distance_to_tp = take_profit - entry_price
            elif position.type == mt5.ORDER_SELL:
                current_price = mt5.symbol_info_tick(symbol).ask
                distance_to_tp = entry_price - take_profit

            # حساب الأهداف المختلفة
            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp) if position.type == mt5.ORDER_BUY else entry_price - (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp) if position.type == mt5.ORDER_BUY else entry_price - (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp) if position.type == mt5.ORDER_BUY else entry_price - (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if (position.type == mt5.ORDER_BUY and new_stop_loss > current_stop_loss) or \
               (position.type == mt5.ORDER_SELL and new_stop_loss < current_stop_loss):
                update_stop_loss(position, new_stop_loss)


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # تحديد السعر الحالي بناءً على نوع الصفقة
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # حساب المسافة إلى جني الأرباح
            distance_to_tp = take_profit - entry_price if position.type == 0 else entry_price - take_profit

            # حساب الأهداف المختلفة
            fifty_percent_target = entry_price + (0.36 * distance_to_tp) if position.type == 0 else entry_price - (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp) if position.type == 0 else entry_price - (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp) if position.type == 0 else entry_price - (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            # تحديث وقف الخسارة بناءً على السعر الحالي
            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp) if position.type == 0 else entry_price - (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp) if position.type == 0 else entry_price - (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp) if position.type == 0 else entry_price - (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if (position.type == 0 and new_stop_loss > current_stop_loss) or \
               (position.type == 1 and new_stop_loss < current_stop_loss):
                update_stop_loss(position, new_stop_loss)


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة7777 بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # تحديد السعر الحالي بناءً على نوع الصفقة
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # حساب المسافة إلى جني الأرباح (مع مراعاة نوع الصفقة)
            distance_to_tp = take_profit - entry_price if position.type == 0 else entry_price - take_profit

            # حساب الأهداف المختلفة (مع مراعاة نوع الصفقة)
            fifty_percent_target = entry_price + (0.36 * distance_to_tp) if position.type == 0 else entry_price - (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp) if position.type == 0 else entry_price - (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp) if position.type == 0 else entry_price - (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            # تحديث وقف الخسارة بناءً على السعر الحالي (مع مراعاة نوع الصفقة)
            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp) if position.type == 0 else entry_price - (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp) if position.type == 0 else entry_price - (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp) if position.type == 0 else entry_price - (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم (مع مراعاة نوع الصفقة)
            if (position.type == 0 and new_stop_loss > current_stop_loss) or \
               (position.type == 1 and new_stop_loss < current_stop_loss):
                update_stop_loss(position, new_stop_loss)

In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # تحديد السعر الحالي بناءً على نوع الصفقة
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # حساب المسافة إلى جني الأرباح (مع مراعاة نوع الصفقة)
            distance_to_tp = abs(take_profit - entry_price)  # استخدام المسافة المطلقة دائمًا

            # حساب الأهداف المختلفة (مع مراعاة نوع الصفقة)
            fifty_percent_target = entry_price + (0.36 * distance_to_tp) if position.type == 0 else entry_price - (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp) if position.type == 0 else entry_price - (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp) if position.type == 0 else entry_price - (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            # تحديث وقف الخسارة بناءً على السعر الحالي (مع مراعاة نوع الصفقة)
            if current_price >= fifty_percent_target and current_price < seventy_five_percent_target:  # إضافة فحص النطاق
                new_stop_loss = entry_price + (0.12 * distance_to_tp) if position.type == 0 else entry_price - (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target and current_price < ninety_percent_target:  # إضافة فحص النطاق
                new_stop_loss = entry_price + (0.24 * distance_to_tp) if position.type == 0 else entry_price - (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp) if position.type == 0 else entry_price - (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم (مع مراعاة نوع الصفقة)
            if (position.type == 0 and new_stop_loss > current_stop_loss) or \
               (position.type == 1 and new_stop_loss < current_stop_loss):
                # فرض تحديث إيقاف الخسارة حتى لو كان التغيير صغيرًا جدًا
                if abs(new_stop_loss - current_stop_loss) > mt5.symbol_info(symbol).trade_tick_size:
                    update_stop_loss(position, new_stop_loss)

In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)



In [ ]:
def update_stop_loss(position, new_stop_loss):
    """تحديث وقف الخسارة للصفقة المحددة مع الحفاظ على تيك بروفت."""
    request = {
        "action": mt5.TRADE_ACTION_SLTP,
        "position": position.ticket,
        "sl": new_stop_loss,
        "tp": position.tp,  # الحفاظ على تيك بروفت كما هو
        "symbol": position.symbol,
        "magic": position.magic,
        "deviation": 10,
        "comment": "Update Stop Loss",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    return result


def set_break_even():
    """تعيين وقف الخسارة إلى مستويات مختلفة لجميع الصفقات المفتوحة، مع عدم التراجع."""
    positions = mt5.positions_get()

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp  # سعر جني الأرباح

            # حساب المسافة بين سعر الدخول وسعر جني الأرباح
            distance_to_tp = take_profit - entry_price

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(position.symbol).bid  # الحصول على السعر الحالي

            # حساب النسب المئوية
            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.72 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            # تعيين وقف الخسارة بناءً على تقدم السعر
            new_stop_loss = position.sl  # الحصول على وقف الخسارة الحالي

            if current_price >= fifty_percent_target:
                new_stop_loss = max(new_stop_loss, entry_price + (0.12 * distance_to_tp))
                update_stop_loss(position, new_stop_loss)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = max(new_stop_loss, entry_price + (0.24 * distance_to_tp))
                update_stop_loss(position, new_stop_loss)

            if current_price >= ninety_percent_target:
                new_stop_loss = max(new_stop_loss, entry_price + (0.48 * distance_to_tp))
                update_stop_loss(position, new_stop_loss)

In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف لكل صفقة."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # تحديد السعر الحالي بناءً على نوع الصفقة
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # حساب المسافة إلى جني الأرباح
            distance_to_tp = take_profit - entry_price if position.type == 0 else entry_price - take_profit

            # حساب الأهداف المختلفة
            fifty_percent_target = entry_price + (0.36 * distance_to_tp) if position.type == 0 else entry_price - (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp) if position.type == 0 else entry_price - (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp) if position.type == 0 else entry_price - (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            # تحديث وقف الخسارة بناءً على السعر الحالي
            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp) if position.type == 0 else entry_price - (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp) if position.type == 0 else entry_price - (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp) if position.type == 0 else entry_price - (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if (position.type == 0 and new_stop_loss > current_stop_loss) or \
               (position.type == 1 and new_stop_loss < current_stop_loss):
                update_stop_loss(position, new_stop_loss)


In [ ]:
# prompt: انشئ دالة بريك ايفن تعمل على جميع الصفقات بيع او شراء و في نفس اللحظة

def set_break_even(symbol):
    """
    Sets the stop-loss to different levels based on price progress towards the target,
    without moving the stop-loss if the new one is lower than the old one.
    Works for both buy and sell orders simultaneously.
    """
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl

            # Determine the current price based on the order type
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # Calculate the distance to take profit (absolute value)
            distance_to_tp = abs(take_profit - entry_price)

            # Calculate targets (adjust for buy/sell)
            fifty_percent_target = entry_price + (0.36 * distance_to_tp) if position.type == 0 else entry_price - (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp) if position.type == 0 else entry_price - (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp) if position.type == 0 else entry_price - (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss

            # Update stop-loss based on current price and order type
            if position.type == 0 and current_price >= fifty_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.12 * distance_to_tp))
            elif position.type == 1 and current_price <= fifty_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.12 * distance_to_tp))


            if position.type == 0 and current_price >= seventy_five_percent_target :  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.24 * distance_to_tp))
            elif position.type == 1 and current_price <= seventy_five_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.24 * distance_to_tp))


            if position.type == 0 and current_price >= ninety_percent_target: #Buy Order
                new_stop_loss = max(new_stop_loss, entry_price + (0.48 * distance_to_tp))
            elif position.type == 1 and current_price <= ninety_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.48 * distance_to_tp))


            # Check if the new stop-loss is different and valid before updating
            if abs(new_stop_loss - current_stop_loss) > mt5.symbol_info(symbol).trade_tick_size:
                update_stop_loss(position, new_stop_loss)